In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.optimizers import Adam

In [5]:
from tensorflow.keras.applications import VGG19, VGG16

In [6]:
import cv2
import os 
import pickle

In [7]:
from imutils import paths

In [8]:
LABELS = set(["Crown and Root Rot", "Healthy Wheat", "Leaf Rust", "Wheat Loose Smut"])

In [9]:
count = 0 
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [10]:
image = cv2.imread('Images/Crown and Root Rot/00041.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

In [11]:
count = 0
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [12]:
sample_path = list(paths.list_images('Images/Crown and Root Rot/'))[0]

In [13]:
sample_path

'Images/Crown and Root Rot/00041.jpg'

In [14]:
sample_path.split('/')

['Images', 'Crown and Root Rot', '00041.jpg']

In [15]:
os.listdir()

['.git',
 '.gitattributes',
 '.ipynb_checkpoints',
 'Images',
 'README.md',
 'Wheat Disease Detection.ipynb']

In [16]:
Labels = os.listdir('Images')

In [17]:
print(Labels)

['Crown and Root Rot', 'Healthy Wheat', 'Leaf Rust', 'Wheat Loose Smut']


In [18]:
CROWN_AND_ROOT_ROT_PATH = 'Images/Crown and Root Rot/'
HEALTHY_AND_WHEAT_PATH = 'Images/Healthy Wheat/'
LEAF_RUST_PATH = 'Images/Leaf Rust/'
WHEAT_LOOSE_SMUT_PATH = 'Images/Wheat Loose Smut/'

In [19]:
from tqdm import tqdm

In [20]:
data = []
labels = []
for image_path in tqdm(paths.list_images(CROWN_AND_ROOT_ROT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(HEALTHY_AND_WHEAT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)

for image_path in tqdm(paths.list_images(LEAF_RUST_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(WHEAT_LOOSE_SMUT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
    

1021it [00:04, 252.24it/s]
1146it [00:21, 53.25it/s] 
1286it [01:27, 14.73it/s] 
930it [00:04, 208.70it/s]


In [21]:
labels[-1]

'Wheat Loose Smut'

In [22]:
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

## VGG19 Network

VGG19 nework trained on imagenet weights is taken as our base model. In addition to this network, we add more layers so that the final model is used for our task of detecting and identifying wheat diseases. 

In [23]:
headmodel = VGG19(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

In [24]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [26]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [27]:
labels[0]

array([1, 0, 0, 0])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2,
                                                    stratify = labels, random_state = 101)

In [29]:
len(X_train)

3506

In [30]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12394139845768866608
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7009950106
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3718247561560573176
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [31]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 30, batch_size = 64)

Train on 3506 samples, validate on 877 samples
Epoch 1/30
3506/3506 [==============================] - 25s 7ms/sample - loss: 1.4687 - accuracy: 0.6937 - val_loss: 0.6549 - val_accuracy: 0.7845
Epoch 2/30
3506/3506 [==============================] - 13s 4ms/sample - loss: 0.4309 - accuracy: 0.8557 - val_loss: 0.3612 - val_accuracy: 0.8871
Epoch 3/30
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.2547 - accuracy: 0.9102 - val_loss: 0.2967 - val_accuracy: 0.9122
Epoch 4/30
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.1681 - accuracy: 0.9492 - val_loss: 0.2186 - val_accuracy: 0.9384
Epoch 5/30
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.1227 - accuracy: 0.9655 - val_loss: 0.1959 - val_accuracy: 0.9453
Epoch 6/30
3506/3506 [==============================] - 14s 4ms/sample - loss: 0.0933 - accuracy: 0.9769 - val_loss: 0.1991 - val_accuracy: 0.9510
Epoch 7/30
3506/3506 [==============================] - 14s 4ms/sample 

In [21]:
from tensorflow.keras.applications import Xception, ResNet152, InceptionV3, NASNetLarge

In [33]:
headmodel = Xception(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

In [34]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

In [35]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 30, batch_size = 64)

Train on 3506 samples, validate on 877 samples
Epoch 1/30
3506/3506 [==============================] - 17s 5ms/sample - loss: 0.7773 - accuracy: 0.7151 - val_loss: 27.5259 - val_accuracy: 0.2440
Epoch 2/30
3506/3506 [==============================] - 10s 3ms/sample - loss: 0.4292 - accuracy: 0.8574 - val_loss: 39.5662 - val_accuracy: 0.3022
Epoch 3/30
3506/3506 [==============================] - 11s 3ms/sample - loss: 0.3127 - accuracy: 0.8990 - val_loss: 55.7753 - val_accuracy: 0.2953
Epoch 4/30
3506/3506 [==============================] - 11s 3ms/sample - loss: 0.2099 - accuracy: 0.9373 - val_loss: 65.4434 - val_accuracy: 0.3044
Epoch 5/30
3506/3506 [==============================] - 11s 3ms/sample - loss: 0.1881 - accuracy: 0.9481 - val_loss: 63.2203 - val_accuracy: 0.3181
Epoch 6/30
3506/3506 [==============================] - 11s 3ms/sample - loss: 0.1249 - accuracy: 0.9675 - val_loss: 74.6631 - val_accuracy: 0.3113
Epoch 7/30
3506/3506 [==============================] - 11s 3ms/s

## InceptionV3 Network

In the below cells, InceptionV3 network would be implemented with the weights taken from imagenet data. Inception weights are not trained and additional layers that are added on top of inception are trained to help us get the output for our prediction of disease in wheat. 

In [ ]:
headmodel = InceptionV3(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

87916544/87910968 [==============================] - 2s 0us/step


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SUHASM~1\AppData\Local\Temp/ipykernel_1144/2610945554.py", line 2, in <module>
    input_tensor = Input(shape = (224, 224, 3)))
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\tensorflow_core\python\keras\applications\__init__.py", line 49, in wrapper
    return base_fun(*args, **kwargs)
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\tensorflow_core\python\keras\applications\inception_v3.py", line 32, in InceptionV3
    return inception_v3.InceptionV3(*args, **kwargs)
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\keras_applications\inception_v3.py", line 391, in InceptionV3
    model.load_weights(weights_path)
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 181, in load_weights
    return super(Model,

Traceback (most recent call last):
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SUHASM~1\AppData\Local\Temp/ipykernel_1144/2610945554.py", line 2, in <module>
    input_tensor = Input(shape = (224, 224, 3)))
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\tensorflow_core\python\keras\applications\__init__.py", line 49, in wrapper
    return base_fun(*args, **kwargs)
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\tensorflow_core\python\keras\applications\inception_v3.py", line 32, in InceptionV3
    return inception_v3.InceptionV3(*args, **kwargs)
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\keras_applications\inception_v3.py", line 391, in InceptionV3
    model.load_weights(weights_path)
  File "C:\Anaconda\envs\gpu_test\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 181, in load_weights
    return super(Model,

In [36]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

ERROR! Session/line number was not unique in database. History logging moved to new session 786


In [ ]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 30, batch_size = 64)

## ResNet152 Network

ResNet152 is used to check the performance of the models. 

In [ ]:
headmodel = ResNet152(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

In [ ]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

In [ ]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 30, batch_size = 64)

In [ ]:
headmodel = NasNetLarge(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False